In [ ]:
import numpy as np
import pandas as pd
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
import warnings
warnings.filterwarnings('ignore')
import os

from typing import List
from tqdm import tqdm

# Assuming the dataset was downloaded from Kaggle, tar file unzipped,
# and is on your Desktop in a folder called archive

# Set up the home directory 
home_directory = os.path.expanduser("~")

# File paths for the dataset
train_file_path = os.path.join(home_directory, 'Desktop/archive/sign_mnist_train.csv')
test_file_path = os.path.join(home_directory, 'Desktop/archive/sign_mnist_test.csv')

# Load the datasets
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

# Vectorize data, and  normalize

x_train = train_df.iloc[:, 1:].values / 255.0
x_test = test_df.iloc[:, 1:].values / 255.0

# Reshape the pixel values into flat vectors
x_train_mlp = x_train.reshape(-1, 28*28)
x_test_mlp = x_test.reshape(-1, 28*28)


# Now separate our dataframes into inputs (features) and outputs (labels)
y_train = train_df['label'].values  # Convert to numpy array
y_test = test_df['label'].values    # Convert to numpy array

# Convert labels to one-hot encoded format
num_classes = 26  # labels 0-25 classes 
y_train_encoded = np.eye(num_classes)[y_train]
y_test_encoded = np.eye(num_classes)[y_test]


In [ ]:
import numpy as np
from tqdm import tqdm


class NeuralNetLayer:
    def __init__(self):
        self.gradient = None
        self.parameters = None
        
    def forward(self, x):
        raise NotImplementedError

    def backward(self, gradient):
        raise NotImplementedError
    
class LinearLayer(NeuralNetLayer):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.ni = input_size
        self.no = output_size
        self.w = np.random.randn(output_size, input_size)
        self.b = np.random.randn(output_size)
        self.cur_input = None
        self.parameters = [self.w, self.b]

    def forward(self, x):
        self.cur_input = x
        return (self.w[None, :, :] @ x[:, :, None]).squeeze() + self.b

    def backward(self, gradient):
        assert self.cur_input is not None, "Must call forward before backward"
        #dw = gradient.dot(self.cur_input)
        dw = gradient[:, :, None] @ self.cur_input[:, None, :]
        db = gradient
        self.gradient = [dw, db]
        return gradient.dot(self.w)
    
class ReLULayer(NeuralNetLayer):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        self.gradient = np.where(x > 0, 1.0, 0.0)
        return np.maximum(0, x)

    def backward(self, gradient):
        assert self.gradient is not None, "Must call forward before backward"
        return gradient * self.gradient
    

class SoftmaxOutputLayer(NeuralNetLayer):
    def __init__(self):
        super().__init__()
        self.cur_probs = None

    def forward(self, x):
        exps = np.exp(x)
        probs = exps / np.sum(exps, axis=-1)[:, None]
        self.cur_probs = probs
        return probs

    def backward(self, target):
        assert self.cur_probs is not None, "Must call forward before backward"
        return self.cur_probs - target
    
class MLP:
    def __init__(self, activation_function, hidden_layer_sizes):
        self.layers = []
        input_size = 28 * 28  # Assuming MNIST-like input size
        for output_size in hidden_layer_sizes:
            self.layers.append(LinearLayer(input_size, output_size))
            self.layers.append(activation_function())
            input_size = output_size
        self.layers.append(LinearLayer(input_size, 26))  # Output layer
        self.layers.append(SoftmaxOutputLayer())

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, target):
        for layer in self.layers[::-1]:
            target = layer.backward(target)

    def fit(self, X, y, learning_rate, num_iterations):
        labels = np.eye(26)[np.array(y)]
        for _ in tqdm(range(num_iterations)):
            predictions = self.forward(X)
            loss = -(labels * np.log(predictions)).sum(axis=-1).mean()
            self.backward(labels)
            self.update_weights(learning_rate)
        return loss

    def update_weights(self, learning_rate):
        for layer in self.layers:
            if layer.parameters is not None:
                for param, grad in zip(layer.parameters, layer.gradient):
                    param -= learning_rate * grad.mean(axis=0)

    def predict(self, X):
        predictions = self.forward(X)
        return np.argmax(predictions, axis=1)


def evaluate_acc(self,y, y_pred):
        accuracy = sum(y_pred.argmax(axis=1) == y.argmax(axis=1))
        accuracy = accuracy / y.shape[0]
        return accuracy


# Example usage

# Initialize MLP
mlp = MLP(activation_function=ReLULayer, hidden_layer_sizes=[64, 64])

# Train MLP
mlp.fit(x_train, y_train, learning_rate=0.1, num_iterations=200)

# Predictions
y_pred_train = mlp.predict(x_train)
y_pred_test = mlp.predict(x_test)

# Evaluate accuracy
accuracy_train = evaluate_acc(y_train, y_pred_train)
accuracy_test = evaluate_acc(y_test, y_pred_test)

print("Training Accuracy:", accuracy_train)
print("Test Accuracy:", accuracy_test)

